In [93]:
import re

import pandas as pd

In [3]:
embedding_path = 'embedding/glove.twitter.27B.200d.txt'

In [78]:
df = pd.read_csv(embedding_path, sep=' ', index_col=0, header=None, quoting=3)
df.head()

Time: 37.02990484237671


,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
0,,,,,,,,,,,,,,,,,,,,,
<user>,0.31553,0.537650,0.101770,0.032553,0.003798,0.015364,-0.203440,0.33294,-0.20886,0.100610,...,-0.062927,0.26674,-0.236170,0.181090,-0.265830,0.090904,-0.81205,-0.456640,-0.465400,0.52066
.,0.35132,0.000561,-0.214880,-0.047070,-0.177770,0.661620,-0.007481,-0.15963,-0.22129,0.655230,...,-0.443700,0.14186,-0.424750,0.023551,-0.096965,0.079513,-1.46830,0.036684,-0.037206,0.85384
:,0.80767,0.497860,0.082696,-0.007930,0.082471,-0.593600,-0.187530,0.48645,0.10719,-0.312990,...,-0.155520,0.11186,0.068971,-0.550480,-0.500100,0.188600,-0.85631,-0.073302,-0.477850,0.67059
rt,0.55687,0.632840,-0.156090,0.263970,0.280150,-0.365060,-0.121280,0.45217,-0.16123,0.015791,...,-0.306690,-0.15438,-0.198570,0.064394,-0.143460,-0.105240,-0.57253,0.238570,-0.792350,0.23761
",",0.39270,-0.084181,-0.607500,0.323100,-0.359190,0.626640,0.297580,-0.21039,-0.23201,0.118970,...,-0.618590,-0.32079,-0.251190,0.278930,0.257740,-0.466150,-0.48912,0.093868,-0.585060,0.19544


In [207]:
'degr' in df.index

True

In [132]:
df.index.isna().sum()

2

In [138]:
df.index[3863] == df.index[3863]

False

In [139]:
cnt = 0
for i, x in enumerate(df.index):
    if x != x:
        print(i, x)
        continue
    if x[0] == '<' and x[-1] == '>':
        print(i, x)
        cnt += 1
print(cnt)   

0 <user>
5 <repeat>
6 <hashtag>
7 <number>
8 <url>
18 <allcaps>
19 <elong>
23 <smile>
103 <sadface>
136 <heart>
141 <lolface>
227 <neutralface>
3863 nan
33097 nan
39030 <b>
104614 <br>
125051 <amazon>
129693 <o>
165987 <p>
191979 <i>
252740 <retweet>
267147 <em>
287564 <n>
287948 <_>
316204 <bot>
336771 <strong>
340741 <spasi>
350316 <censored>
374372 <hr>
396424 <pic>
410055 <rt>
410056 <unknown>
422608 <space>
518509 <preview>
521428 <!--beg_competition_id_<number>
521907 <!--end_competition_id_<number>
535262 <div>
547437 <head>
562729 <info>
590763 <html>
595377 <time>
598750 <sigh>
623192 <script>
635471 <myroom>
635472 <pressure>
635473 <temp>
716682 <title>
719826 <m>
751878 <s>
820796 <c>
826640 <g>
840169 <candy>
921688 <stdio.h>
933530 <u>
945520 <a>
947511 <jarak>
957978 <update>
960054 <body>
985953 <style>
992816 <center>
995064 <dmm>
995065 <nn>
999571 <fm-fuji>
1020975 <system>
1133846 <lol>
1143379 <poison>
1146547 <!--tbegin:<url>
1190023 <span>
66


In [155]:
'' in df.index

True

In [94]:
a = 'Box A contains 3 red and 5 white balls, while Box B contains 4 red and 2 blue balls.'

a = re.sub('\d+', '<number>', a)

a

'Box A contains <number> red and <number> white balls, while Box B contains <number> red and <number> blue balls.'

In [1]:
import nltk
import contractions

In [2]:
s = "At eight o'clock on Thursday morning, Arthur didn't feel very good."

s = "  He's   the   man,  the point is three hundred  degree.  "

s2 = contractions.fix(s)
print(s2)
tokens = nltk.word_tokenize(s2)
print(tokens)
tagged = nltk.pos_tag(tokens)
print(tagged)

  he is   the   man,  the point is three hundred  degree.  
['he', 'is', 'the', 'man', ',', 'the', 'point', 'is', 'three', 'hundred', 'degree', '.']
[('he', 'PRP'), ('is', 'VBZ'), ('the', 'DT'), ('man', 'NN'), (',', ','), ('the', 'DT'), ('point', 'NN'), ('is', 'VBZ'), ('three', 'CD'), ('hundred', 'JJ'), ('degree', 'NN'), ('.', '.')]


# Data Clean

In [2]:
from dataset import DataBuilder

In [3]:
convs = DataBuilder.load(data_dir='data')
len(convs)

 Load: 100%|██████████| 83097/83097 [00:46<00:00, 1796.58it/s]


83097

In [4]:
convs = DataBuilder.clean(convs)
pairs = DataBuilder.make_pairs(convs)

Clean: 100%|██████████| 83097/83097 [01:16<00:00, 1084.60it/s]


83097 conversations cleaned.
221616 pairs built.


In [8]:
max_len = 30

def _filter(sent):
    if len(sent) > max_len:
        return False
    exclude = ',.!?'.split() + ['<number>']
    sent = [x for x in sent if x not in exclude]
    if len(sent) <= 3:
        return False
    return True

In [9]:
pairs_filter = [(x, y) for x, y in pairs if _filter(x) and _filter(y)]

len(pairs_filter)

137952

In [10]:
len(pairs_filter) / len(pairs)

0.622482131254061

In [257]:
def clean(sent):
    sent = sent.lower().strip()
    sent = contractions.fix(sent)
    sent = re.sub(r"('ll)", r" will", sent)
    sent = re.sub(r"('s)", r" \1", sent)
    sent = re.sub(r"[^a-zA-Z0-9',.!?]+", r' ', sent)
    sent = re.sub(r'([,.!?])\1*', r' \1 ', sent)
    sent = re.sub(r'\d+\.?\d*', r' <number> ', sent)
    sent = re.sub(r'\s+', r' ', sent)
    sent = nltk.word_tokenize(sent)
    return sent

In [259]:
cnt = 0
for conv in convs:
    for sent in conv:
        sent = clean(sent)
        if "<number>" in sent:
            print(sent)
            
print(cnt)

then guillermo says , if you go any lighter , you are going to look like an extra on <number> . 
in <number> th . for a month
like where ? the <number> eleven on burnside ? do you even know my name , screwboy ? 
i thought we decided you were going to school here . at you of <number> . 
then you cannot ignore that according to their calculations , the circumference of the earth is approximately . <number> , <number> leagues or more . which makes the ocean . uncrossable . 
ignorance ! i believe the indies are no more than <number> leagues west of the canary islands . 
you lied ! you cheated ! we are way past <number> leagues ! 
ladder <number> was on the rock for training . we stopped there . so she could get cleaned up . 
what is the kid going to say sorry ? meanwhile I am not here anymore . like last week we were at the morgue and this guy was all chopped up spleen here liver there his heart in a pan . six hours ago this guy was walkin' his dog or buyin' a quart of milk . who knows ? b

the sun will heat that thing to over <number> degrees within seconds . he will roast like a pig on a spit . 
the hell it is not , when somebody wires up enough c <number> explosive to my car to turn me into pink mist ! that slag was on something , and not sour milk , either ? am i right ? tell me ! what is it ? 
casull . <number> magnum . you are talking twice the impact energy of . <number> magnum hot loads . 
you just try getting a dangerous organism past icc quarantine . section <number> of the commerce code . 
all right , we can not blow the fuck out of them . why not roll some canisters of cn <number> down there . nerve gas the whole nest ? 
why will not you check out lv <number> ? 
not at all . that is alexander hamilton 's idea <number> the finest banking mind this country has ever known . those are his exact words , gentlemen . character ! it is the only thing you can bank on , and it is the only thing that will pull this country out of the doldrums . 
what is that , mr . bones

all pertinent information is being fed into your departmental computer , an esper <number> i believe and a photo over lay packet is being produced . 
all right . let me tell you the whole thing . in <number> 
 <number> . 
 <number> . seward 's folly . we paid <number> . <number> million dollars for it . a tidy sum then , as well as now . I am quoting my father , of course . 
here 's the shopping list and <number> , <number> which should take care of everything . 
uh huh . and it is all yours for the remarkably low price of only <number> ! and if you act now , i might even throw in some free lawn furniture . 
none of this stuff was here in <number> . the valley was mostly small homes and fruit orchards . 
I am marking the <number> radial for five and ejecting . 
I have made arrangements with dr . gynde for <number> <number> . but jeffrey , you will have to walk over i need the car this morning . 
i already know . the slow club . it is on route <number> . 
your dad and i used him three t

you have good taste . french , <number> 's , attributed to chitarus . 
that was taken nine years ago . i . d . would as michael hanover . sentenced to <number> months in the state penn at rawlins , wyoming for armed robbery . he escaped from custody after <number> days . has not been seen or heard from since . 
i got a hit on the car . an oh five tahoe , blue , with wyoming tags . agt <number> . it was not easy because you were off in one of the numbers . 
hot run to <number> beulah , corner of haight . and you can tell your pal theo he just got his worthless ass fired . not that he cares but the wife and kid might . 
for the next hour and a half it is every dude for himself . try to get at least one ticket and at <number> <number> p . m . we will meet over there . 
then lighten up . she will never know we touched it . alright , here 's the plan . we bus it to chez lex , grab the volvo , bail jam the hell outta st . bernard 's and arrive at the train station precisely on time for the <

bad enough we got a full moon it is friday the <number> th . they keep statistics . we get more accidents , more robberies , more rapes , more homicides , more of everything when there is a full moon . it affects people . makes them nuts . 
i said new , this is an ' <number> . 
it is hardly been used at all . i got it from a drug dealer who only drove it <number> times in as many years . swear to god . that is like new . 
shit , i been to bars make this place look like a fuckin' <number> h club . 
the sword of damocles is lifted from above seth 's head . he is just solved a problem that a mere thirty seconds ago seemed unsolvable . he knows exactly how he is going to cross the border . whistling a happy tune , he turns and walks back into room <number> . 
i do not know if there is any single reason . but my father was navy . and he had this old time recruiting poster in his den . it showed a girl trying on a sailor 's uniform while saying , gee , i wish i were a man ! I would join the 

...

give it another ten minutes . if we do not hell , I will tell him . pete ? if we have not got anything new in the next ten minutes just go with the wire service copy under the <number> pt . banner and run the picture for three columns . kent , i need a background story for the page <number> sidebar . give me every thing you know about this terrorist group . 
now you try , clark . we will start you off with <number> pounds . 
lex , we all appreciate how you have supported us lately and . . . uh . . . in recognition of your hard work we have decided to increase your commission to <number> percent . 
know something ? you are a workaholic . a good guy <number> hours a day . stop and smell the roses sometime . take up a hobby . get a pet . . . a kitten or a puppy . . . 
that the planet blew up on may <number> , <number> . that the rocket ship with the revolting little freak in it took three years to reach earth . ergo ? 
it is very simple , otis . very simple , the xk <number> rocket . it i

he says you were bar mitzvahed there , in march <number> . 
take a right on <number> th . we will put it someplace else . 
this is good work . unfortunately it is incomplete . could you eat a sandwich <number> floors above the street . 
tonight we are having a little party among ourselves . <number> of the clock in my suite ? 
santini will do that for <number> dollars ! 
i dove <number> feet ! 
uncle louis . he is a <number> year old amazing guy , is not he ? 
yes sir , and it has been so for <number> years . 
they are paramedics . they wait at <number> feet . 
those are the divers who go down <number> to <number> feet . they dive with a helium oxygen mixture . at that depth , compressed air is too dangerous because it contains carbon dioxide . 
I am going to pay a visit to the k . l . f . see if they have got the <number> on santa 's location . I will check in with you when i know more . 
yeah , I am going to be out of the office from <number> <number> to <number> <number> on sunday .

I will pay you <number> each in advance and I will mount you on those . 
doc will not quit , been up <number> hours . clanton came in an hour ago , they switched over to poker . tried to get him to bed but he just will not let go . 
 <number> . year 's wages . and i never even saw a horse like that . 
talked it over . we decided we do not need the money . took out <number> dollars each , though . federal posseman 's fee . that all right ? 
look at that . busted . brand new money ward , too . <number> years old and i do not even have a decent watch . 'bout time i started lookin' out for myself . 
ah , now that is the real mother lode in tombstone . up and down allen street , full blast <number> hours a day , liquor , hostesses , gamblin' , makin' money hand over fist . all except the oriental . on account of the element . have a man for breakfast in there most days . regular slaughter house . high rollers will not go near it . too bad , nice place . 
 I will be sure to acknowledge <numb

dunne , it is jenkins , we got a guy at gate <number> c with a bogus ticket . says he knows you . he will not tell me his name . 
do not matter . they burned the table in ' <number> for firewood . 
he said how you was really three fingered jack out of missouri . an' bill said same three fingered jack that dynamited the rock island and pacific in ' <number> killin' women and children an' all ? an' ned says you done a lot worse than that , said you was more cold blooded than william bonney or clay alisson or the james brothers an' how if he hurt ned again you was going to come an' kill him like you killed a you . s . marshall in ' <number> . 
 lt lt I will tell you everything . i' <number> even say it slow enough for you to understand it . just tell this man i want protection . real protection . gt gt 
i' <number> have this thing in front of a grand jury by monday . 
fuck you . i' <number> never shit right again . so who did it ? own up . 
i' <number> be alright . 
forget it . i' <number

at <number> . <number> . 
i can not believe that is your mother . we are talking babe a licious chub fest , right here in the <number> 's . 
this is not some <number> 's porno film , you dip shit . there are no wild sex orgies . 
i remember the first time you walked into mrs . temple 's class in <number> nd grade . blue gingham dress . blond ponytail . 
at least it is not another <number> h meeting or a big brother barbecue . 
at our press conference tomorrow , i think we should announce our plans to explore the world . not just a few leggy <number> d 's , with blond hair upstairs and down . 
he said something like , do you want it dirty <number> or something . i tried to turn away , but he pushed me to the floor . 
zenovitch was about to deliver a list of names tonight street chemists . the guys processing this p <number> p into speed . 
that is the problem . we need results . the press is driving us crazy over this p <number> p thing . calling us the 'speed capitol of the country' . 

In [249]:
sent = "He's bag, is Tom's."
print(clean(sent))

he is bag, is tom's.
he is bag , is tom's .
he is bag , is tom 's .
he is bag , is tom 's .
